# 【問題1】クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。<br>
事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。<br>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import optuna

from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
#データ読み込み
df = pd.read_csv("/Users/takahashihideyuki/dive/diveintocode-ml/Week3/application_train.csv")
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
#学習から検証までを関数化 　　KFoldを使用
def learning_to_verification(X, y, model):
    X = np.array(X)
    y = np.array(y)
    
    # クロスバリデーションで分割する。
    auc_list = []
    cnt = 1
    kf = KFold(n_splits=5, random_state=0, shuffle=True)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #標準化
        scaler = StandardScaler()
        scaler.fit(X_train) #訓練用のデータでfit
        X_train_std = scaler.transform(X_train) #訓練用データをtransform
        X_test_std = scaler.transform(X_test) #検証用データをtransform

        #学習〜予測
        clf = model
        clf.fit(X_train_std, y_train) # 学習
        pred = clf.predict(X_test_std) #クラスの予測
        pred_proba = clf.predict_proba(X_test_std) #クラスの予測確率
        pred_proba_posi = pred_proba[:, 0] #クラスの予測確率（ポジティブクラスを抜粋）

        # 評価
        # ROC曲線の計算
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_proba_posi, pos_label=0)

        #AUC（Area Under the Curve）を計算
        auc_value = auc(fpr, tpr)
        print("{}回目のAUC値は{:.2f}です。".format(cnt, auc_value))
        auc_list.append(auc_value)
        cnt += 1 
        
    auc_mean = sum(auc_list) / len(auc_list)
    print("AUC平均値は{:.2f}です。".format(auc_mean))

In [43]:
#ベースラインモデル（特徴量は前回の課題で選定したもの、重要度0.01以上の上位26変数を抽出）
base_columns = np.array(['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH',
       'DAYS_REGISTRATION', 'AMT_ANNUITY', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_CREDIT', 'DAYS_EMPLOYED', 'AMT_INCOME_TOTAL',
       'REGION_POPULATION_RELATIVE', 'EXT_SOURCE_1',
       'HOUR_APPR_PROCESS_START', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'YEARS_BEGINEXPLUATATION_AVG',
       'APARTMENTS_AVG', 'LANDAREA_AVG', 'OWN_CAR_AGE',
       'BASEMENTAREA_AVG', 'NONLIVINGAREA_AVG', 'YEARS_BUILD_AVG',
       'COMMONAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'ENTRANCES_AVG',
       'CNT_CHILDREN'])

X = df[base_columns].fillna(df[base_columns].median()) #欠損値を中央値で補完
y = df.loc[:, 'TARGET']

learning_to_verification(X, y, model=SGDClassifier(loss="log")) #学習から検証までを関数で実行

1回目のAUC値は0.71です。
2回目のAUC値は0.72です。
3回目のAUC値は0.71です。
4回目のAUC値は0.71です。
5回目のAUC値は0.72です。
AUC平均値は0.71です。


# 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。<br>
機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。<br>
最適なパラメータを探していくことを パラメータチューニング と呼びます。<br>
パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。<br>
scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。<br>
そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。<br>
どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。<br>
GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。<br>
クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [44]:
#学習から検証までを関数化　　　GridSearchCVを使用
def learn_to_verifi_GS(X, y, model, parameters):
    # 訓練データと検証データの分割。訓練データ75%、検証データ25%として分割する。
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
    
    #標準化
    scaler = StandardScaler()
    scaler.fit(X_train) #訓練用のデータでfit
    X_train_std = scaler.transform(X_train) #訓練用データをtransform
    X_test_std = scaler.transform(X_test) #検証用データをtransform
    
    #学習〜予測
    clf = GridSearchCV(model, parameters, cv=5)    
    clf.fit(X_train_std, y_train) # 学習
    display(pd.DataFrame(clf.cv_results_))
    print(clf.best_estimator_)
    print(clf.best_params_)
    pred = clf.predict(X_test_std) #クラスの予測
    pred_proba = clf.predict_proba(X_test_std) #クラスの予測確率
    pred_proba_posi = pred_proba[:, 0] #クラスの予測確率（ポジティブクラスを抜粋）

    # ROC曲線の計算
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_proba_posi, pos_label=0)

    #AUC（Area Under the Curve）を計算
    auc_value = auc(fpr, tpr)
    print("AUC値は{:.2f}です。".format(auc_value))

In [45]:
learn_to_verifi_GS(X, y , model = SGDClassifier(loss="log"), 
                                  parameters = {'alpha':[0.0001, 0.001, 0.01], "validation_fraction":[0.1, 0.5, 0.9], 
                                                           "early_stopping":[True, False]})

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_early_stopping,param_validation_fraction,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.552974,0.030020,0.004250,0.000925,0.0001,True,0.1,"{'alpha': 0.0001, 'early_stopping': True, 'val...",0.919180,0.919180,0.918703,0.919200,0.919481,0.919149,0.000251,14
1,0.511850,0.047352,0.004344,0.000276,0.0001,True,0.5,"{'alpha': 0.0001, 'early_stopping': True, 'val...",0.919527,0.918789,0.918963,0.918072,0.918462,0.918763,0.000488,17
2,0.384462,0.036354,0.004244,0.000189,0.0001,True,0.9,"{'alpha': 0.0001, 'early_stopping': True, 'val...",0.912242,0.912112,0.918985,0.916490,0.914170,0.914800,0.002628,18
3,0.660929,0.052198,0.003506,0.000273,0.0001,False,0.1,"{'alpha': 0.0001, 'early_stopping': False, 'va...",0.919375,0.919006,0.919418,0.918528,0.918788,0.919023,0.000341,16
4,0.690010,0.107271,0.003545,0.000224,0.0001,False,0.5,"{'alpha': 0.0001, 'early_stopping': False, 'va...",0.919375,0.919093,0.919375,0.919351,0.919351,0.919309,0.000109,12
5,0.614212,0.058586,0.004329,0.001834,0.0001,False,0.9,"{'alpha': 0.0001, 'early_stopping': False, 'va...",0.919418,0.919050,0.919158,0.919113,0.919091,0.919166,0.000131,13
6,0.621478,0.027266,0.005510,0.000420,0.001,True,0.1,"{'alpha': 0.001, 'early_stopping': True, 'vali...",0.919613,0.919440,0.919331,0.919416,0.919720,0.919504,0.000142,6
7,0.527496,0.010388,0.004905,0.000430,0.001,True,0.5,"{'alpha': 0.001, 'early_stopping': True, 'vali...",0.919570,0.919483,0.919310,0.919265,0.919633,0.919452,0.000144,9
8,0.487913,0.044635,0.005231,0.000542,0.001,True,0.9,"{'alpha': 0.001, 'early_stopping': True, 'vali...",0.919440,0.918399,0.918529,0.919481,0.919438,0.919058,0.000486,15
9,0.452795,0.009750,0.003534,0.000205,0.001,False,0.1,"{'alpha': 0.001, 'early_stopping': False, 'val...",0.919570,0.919310,0.919396,0.919416,0.919308,0.919400,0.000096,11


SGDClassifier(alpha=0.01, early_stopping=True, loss='log')
{'alpha': 0.01, 'early_stopping': True, 'validation_fraction': 0.1}
AUC値は0.72です。


##### #覚書：グリッドサーチでscoringを指定しない場合、split(n)_test_scoreにはestimatorのscoreメソッドが使われる。(今回の場合はaccuracy)

# 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

- ベイズ最適化<br>
  https://www.kaggle.com/willkoehrsen/automated-model-tuning<br>
  自動ハイパーパラメーター調整：勾配降下法、ベイジアン最適化、または進化アルゴリズムなどの方法を使用して、<br>
  最適なハイパーパラメーターのガイド付き検索を実行します。<br>
  https://towardsdatascience.com/an-introductory-example-of-bayesian-optimization-in-python-with-hyperopt-aae40fff4ff0<br>
  最近では、機械学習モデルのベイズハイパーパラメーター最適化が、手動、ランダム、またはグリッド検索よりも効率的であることを示唆しています。<br>
　　- テストセット全体のパフォーマンスが向上<br>
　　- 最適化に必要な時間の短縮<br>

  optuna  https://optuna.org/
  

- LightGBM<br>
　　「Kaggler」の上位6割以上が LightGBM を用いている。<br>
  https://rightcode.co.jp/blog/information-technology/lightgbm-useful-for-kaggler<br>
  Optuna の拡張機能 LightGBM Tuner によるハイパーパラメータ自動最適化<br>
  https://tech.preferred.jp/ja/blog/hyperparameter-tuning-with-optuna-integration-lightgbm-tuner/

# 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。<br>
その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

In [7]:
def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    alpha = trial.suggest_float('alpha', 0.001, 0.01)
    early_stopping = trial.suggest_categorical("early_stopping", [True, False])
    
    clf = SGDClassifier(loss="log", 
        alpha=alpha, early_stopping=early_stopping)
    
    return cross_val_score(clf, X, y, cv=5).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-08-05 21:32:04,339] Trial 0 finished with value: 0.7687069712457136 and parameters: {'alpha': 0.004847176317116307, 'early_stopping': True}. Best is trial 0 with value: 0.7687069712457136.
[I 2020-08-05 21:32:06,841] Trial 1 finished with value: 0.9192224024313355 and parameters: {'alpha': 0.006809933020186432, 'early_stopping': True}. Best is trial 1 with value: 0.9192224024313355.


KeyboardInterrupt: 